## Prepapre DB

In [135]:
import pandas as pd 
from tqdm import tqdm 
import numpy as np

n = 2508110

features = np.ones((n, 768), dtype=float)
features = features * np.arange(n).reshape(-1, 1)


# df from dict 
df = pd.DataFrame.from_dict({
    'index':  [x for x in range(n)], \
    'field1': [f'string{x}' for x in range(n)], \
    'field2': [x for x in range(n)], \
    'field3': [x for x in range(n)], \
    'feature': features.tolist() \
})
df.head()

: 

: 

In [123]:
document_structure = {  
    "mappings": {
        "properties": {
            "index": {"type": "integer"},
            "field1": {"type": "text"},
            "field2": {"type": "integer"},
            "field3": {"type": "integer"},
            "feature": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "l2_norm"}
        }
    }
}

## Create index in Elasticsearch

In [125]:
from elasticsearch.helpers import bulk
from elasticsearch import Elasticsearch, RequestsHttpConnection

es = Elasticsearch(['http://0.0.0.0:20542'], timeout=100, connection_class=RequestsHttpConnection, http_auth=('elastic', '123456'), use_ssl=False, verify_certs=False)

In [124]:
# Delete index
# es.indices.delete(index='test')

{'acknowledged': True}

In [126]:
es.indices.create(index='test', ignore=[400], **document_structure)
es.indices.get_alias("*")

/tmp/ipykernel_909604/734342283.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.get_alias("*")


{'test': {'aliases': {}}}

In [127]:
for i, row in df.iterrows():
    print(row['index'])
    data = row.to_dict()
    data.pop('index', None)
    print(data)
    break

0
{'field1': 'string0', 'field2': 0, 'field3': 0, 'feature': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [128]:
def gen_data():
    for i, row in tqdm(df.iterrows(), total=len(df)):
        data = row.to_dict()
        index = data.pop('index', None)
        yield {
            "_index": 'test',
            "_id": index,
            "_source": data
        }
bulk(es, gen_data())

100%|██████████| 10/10 [00:00<00:00, 5770.92it/s]


(10, [])

In [53]:
# Delete index
# es.indices.delete(index='test')

{'acknowledged': True}

In [129]:
from elasticsearch_dsl import Search

def search(query):
    query = query.upper()
    global es
    s = Search(using=es, index="test") \
        .query("multi_match", query=query, fields=['field1']) \
        .extra(size=100, explain=True)
    response = s.execute()
    return response
    
responses = search("string3")
for hit in responses[:10]:
    print(hit.meta.score)
    print(hit.meta.id)

1.9924302
3


In [130]:
import logging 
# set logging level to logging.DEBUG to see the detailed output
logging.basicConfig(level=logging.ERROR)

In [134]:
query = np.ones((768), dtype=float) * 7
from elasticsearch import RequestError

def search(query_vector, index, top_k=10):
    # script_query = {
    #     "script_score": {
    #         "query": {"match_all": {}},
    #         "script": {
    #             "source": "l2norm(params.query_vector, 'feature')",
    #             "params": {"query_vector": query_vector.tolist()}
    #         }
    #     }
    # }
    # script_query sort by score, lower is better
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "1 / (1 + l2norm(params.query_vector, 'feature'))",
                "params": {"query_vector": query_vector.tolist()}
            }
        }
    }
    return es.search(index=index, query=script_query, size=top_k)['hits']['hits']

print(query[:10])

responses = search(query, index='test', top_k=10)
for hit in responses[:10]:
    print(hit['_id'],hit['_score'])
    print(hit['_source']['feature'][:10])

# try:
#     responses = search(query, index='test', top_k=1)    
#     # print(responses)
# except RequestError as e:
#     print(e.info['error'])

[7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]
7 1.0
[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
6 0.034827657
[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
8 0.034827657
[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0]
5 0.017722445
[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
9 0.017722445
[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0]
4 0.011885175
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]
3 0.008940445
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
2 0.0071651684
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
1 0.0059781126
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
0 0.0051284763
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
